In [2]:
from dcicutils import ff_utils
from uuid import UUID
import ast
import json
import copy

env = 'data'
my_auth = ff_utils.get_authentication_with_server({}, ff_env = env)

schema_name = {}
profiles = ff_utils.get_metadata('/profiles/', key=my_auth, frame = 'raw')
for key, value in profiles.iteritems():
    schema_name[key] = value['id'].split('/')[-1][:-5]

def is_uuid(value):
    #md5 qualifies as uuid
    if '-' not in value:
        return False
    try:
        UUID(value, version=4)
        return True
    except:
        return False

def find_uuids(val):
    vals = []
    if not val:
        return []
    elif isinstance(val, basestring):
        if is_uuid(val):
            vals = [val]
        else:
            return []
    else:
        text =str(val)
        text_list = [i for i in text. split("'") if len(i) == 36]
        vals = [i for i in text_list if is_uuid(i)]
    return vals
    

In [3]:

def record_object(uuid, store, item_uuids):
    #keep list of fields that only exist in frame embedded (revlinks) that you want connected
    
    add_from_embedded = {'file_fastq':['workflow_run_inputs', 'workflow_run_outputs'],
                         'file_processed':['workflow_run_inputs', 'workflow_run_outputs']
                        }
#    add_from_embedded = {}
   
    #find schema name, store as obj_key, create empty list if missing in store

    obj_type = ff_utils.get_metadata(uuid, key=my_auth, frame='object')['@type'][0]

    try:
        obj_key = schema_name[obj_type]
    except:
        #print 'CAN NOT FIND', obj_type, uuid
        return store, item_uuids
    if obj_key not in store:
        store[obj_key] = []

    add_uuids = []
    resp = ff_utils.get_metadata(uuid, key=my_auth, frame = 'raw')
    
    if resp['status'] not in ['current', 'released', 'in review by lab', 'uploaded']:
        print obj_key, uuid, resp['status']
        
    # add raw frame to store and uuid to list
    if uuid not in item_uuids:
        store[obj_key].append(resp)
        item_uuids.append(uuid)
    # this case should not happen actually
    else:
        return store, item_uuids
    
    fields_to_check = copy.deepcopy(resp)
  
    # check if any field from the embedded frame is required
    add_fields = add_from_embedded.get(obj_key)
    if add_fields:
        emb_resp = ff_utils.get_metadata(uuid, key=my_auth, frame = 'embedded')
        for a_field in add_fields:
            field_val = emb_resp.get(a_field)
            if field_val:
                fields_to_check[a_field] = field_val
        
    for key, value in fields_to_check.iteritems():
        # uuid formatted fields to skip
        if key in ['uuid', 'blob_id', "attachment"]:
            continue
        uuid_in_val = find_uuids(value)
        
        for a_uuid in uuid_in_val:
            if a_uuid not in item_uuids:
                store, item_uuids = record_object(a_uuid, store, item_uuids)   
    return store, item_uuids
            

set_to_release = '46db06ad-b399-4cf4-9acc-07b3e25ef132'
store_dict = {} 
item_uuids = []

store, uuids = record_object(set_to_release, store_dict, item_uuids)

print(len(uuids))
print(len(list(set(uuids))))
    


# for a_type in store:
#     filename = 'Jsons/' + a_type + '.json'
#     with open(filename, 'w') as outfile:
#         json.dump(store[a_type], outfile, indent=4)



software 42adeb1b-c46c-4ba8-b191-56036ddcd544 released to project
69
69


In [9]:
## This part is only for releasing
counter = 0
for a_type in store:
    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            print 'deleted data', a_type ,raw_data['uuid']
            continue        
        if raw_data['status'] not in ['current', 'released', 'released to project']:    
            counter +=1
            print a_type, raw_data['status'], raw_data['uuid']
            patch_data = {'status': "released to project"}
            ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth)
        else:
            # print 'ALREADY RELEASED/TO PROJECT', a_type, raw_data['status'], raw_data['uuid']
            pass
            
print counter

experiment_set_replicate in review by lab 46db06ad-b399-4cf4-9acc-07b3e25ef132
quality_metric_fastqc in review by lab 40144a2d-1553-4fef-b195-f661001e66c4
quality_metric_fastqc in review by lab bd65a647-4b97-4761-94f8-e71c30b15797
quality_metric_fastqc in review by lab a0cd0a5c-505e-4250-85c7-b2da8166a1e4
quality_metric_fastqc in review by lab 6dc35719-6cf4-4650-b9df-fca4ba65cce7
protocol in review by lab fa6738b1-ffc5-4fe6-b960-a1c3bbb26c2d
experiment_seq in review by lab 0ba3f165-7f78-4d57-a6bd-e7403bfd8631
experiment_seq in review by lab 9bba36e4-069b-4cc2-b8e7-b121508803c1
file_processed uploaded 6e6334b9-c368-461f-a2ff-66c21dd2629a
file_processed uploaded 6ada607d-e33b-4524-9542-6c5131c7fd18
file_fastq uploaded 481454f2-e2ae-4dec-9d84-6420602ee527
file_fastq uploaded 5c693443-07ad-4bfb-92ec-e7cf6a823a9d
file_fastq uploaded b00e038a-dec1-41c3-9446-f28118506651
file_fastq uploaded 7571a09e-49d2-4172-9d3a-b8d568b51af9
workflow_run_awsem in review by lab 1ed1384a-8164-4d85-be70-a5b162